# Singular Value Decomposition
## A Python Lab for Undergraduate Students

In [ ]:
# import numpy to deal with matrices
import numpy as np
# import graphing library
import matplotlib.pyplot as plt
# import library that deals with image files
from PIL import Image
# import library that loads built-in datasets
from sklearn import datasets, model_selection
# import library to analyze data sets
import pandas as pd


**Application: Data Compression & De-Noising**

In [ ]:
# load image
img = Image.open("D:\phyll.png")
# turn color image to gray: two-dimensional matrix v. three-dimensional
gray = img.convert('LA')

We like to convert color images into gray scale because the gray scale is encoded by the integers 0:255, AND each element in the image corresponds to an element in an M-by-N matrix.  Color images are M-by-N-by-3 matrices because each pixel has a red-green-blue encoding;  It's easier not to deal with this.

In [ ]:
# display image so you can see it
plt.imshow(gray)

I've created a Dwarven Bard whose primary instruments are the maracas and the kazoo using HeroForge.

In [ ]:
# convert the image object to a numpy array to use the SVD function
A = np.array(list(gray.getdata(band = 0)), float)
A.shape = (gray.size[1], gray.size[0])
A = np.matrix(A)

In [ ]:
# display matrix so you can see
A

We use numpy's svd function in its linear algebra sub-library.  There are other svd functions (a curse and a blessing of open source languages).

In [ ]:
U, S, V = np.linalg.svd(A)

Below we'll check out the magnitudes of the singular values.  Recall they're ordered based on largest, $\sigma_1$, to smallest, $\sigma_r$.  

In [ ]:
plt.plot(S)
plt.show()

Observe the huge drop off in value!  Somewhere between $\sigma_5$ and $\sigma_{20}$.  In the word of data compression, this says to us that the other components don't matter because their singular values are so tiny relative the the first handful.

In [ ]:
for i in range(0,50):
    p = i;
    approx = np.matrix(U[:,:p])*np.diag(S[:p])*np.matrix(V[:p,:]);
    plt.imshow(approx, cmap = 'gray')
    print("Number is ", p)
    plt.show()
    

So, we'll extract the singular vectors and singular values associated only with those first 20 components.

In [ ]:
p = 20
approx = np.matrix(U[:,:p])*np.diag(S[:p])*np.matrix(V[:p,:])

In [ ]:
plt.imshow(approx, cmap = 'gray')

And we get a relatively decent idea of what the image entails.  Not perfect, but it doesn't have to be!

Now we check mathematically how accurate of a prediction the approximate image is to the original.

In [ ]:
res = np.linalg.norm(approx - A)/np.linalg.norm(A)
print(res)

We'll do this process over, but we'll do it with noise added to the original image.  The idea is that the singular values will highlight directions that highlight important features of the image, sifting through the noise.  The largest singular values should be able to indicate to use which components have minimal noise.

In [ ]:
noise_A = np.matrix(A) + 50*np.random.normal(0, 1, A.shape)
plt.imshow(noise_A, cmap = "gray")

In [ ]:
U, S, V = np.linalg.svd(noise_A)
plt.plot(S)
plt.show()

In [ ]:
p = 20
approx_noise = np.matrix(U[:,:p])*np.diag(S[:p])*np.matrix(V[:p,:])
plt.imshow(approx_noise, cmap = 'gray')

In [ ]:
res = np.linalg.norm(approx_noise - A)/np.linalg.norm(A)
print(res)

A little worse than before, but I think we were able to figure out the best representation despite the noise.

**Application: Regression**

In regression, we want to be able to take at least one independent variable $\{{\bf x_1}, {\bf x_2}, \dots, {\bf x_n} \}$ and make inferences about its relationship to some dependent variable, ${\bf y}$.  In linear algebra, we attempt to solve the problem
$$
\min_{{\bf a}}|| {\bf y} - {\bf Xa}||_F,
$$
where ${\bf a}$ will contain the information for the slopes for each ${\bf x_i}$ and for the intercept value.  

We'll use SVD to help give us a decomposition for ${\bf X}$ and then
$$
\begin{array}{l}
{\bf y} = {\bf X a}\\
{\bf y} = {\bf U\Sigma V^{T} a}\\
{\bf V \Sigma^{-1}U^{T}y} = {\bf a}
\end{array}
$$

We'll load a housing dataset for houses in California.

In [ ]:
# import built-in data set from Python
DF = datasets.fetch_california_housing()

In [ ]:
# this code only helps to display the data neatly, don't need it for later.
X = pd.DataFrame(DF['data'], columns = DF['feature_names']); #X = X.style.hide(axis='index')
X.rename({0:"rowindx"})

In [ ]:
# this code only helps to display the data neatly, don't need it for later.
Y = pd.DataFrame(DF['target'], columns=DF['target_names']);#Y = Y.style.hide(axis='index')
Y.rename({0:"rowindx"})

We'll now extract the *observed* data ${\bf X}$ and ${\bf y}$.

In [ ]:
X = DF['data']
y = DF['target']
X = np.column_stack([np.ones(X.shape[0]), X])

Here we split the data into a "training" and a "test" set of ${\bf X}$ and a "training" and "test" set of ${\bf y}$.  

In [ ]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.40)

We'll calculate the SVD of the training ${\bf X}$.

In [ ]:
U, S, V = np.linalg.svd(X_train, full_matrices=False)

Let's see what these singular values look like.

In [ ]:
plt.plot(S)
plt.show()

MUCH more dramatic than before!

We'll use SVD to help give us a decomposition for training ${\bf X}$ and then
$$
\begin{array}{l}
{\bf y} = {\bf X a}\\
{\bf y} = {\bf U\Sigma V^{T} a}\\
{\bf V \Sigma^{-1}U^{T}y} = {\bf a}
\end{array}
$$


In [ ]:
a = V.T @ np.linalg.inv(np.diag(S)) @ U.T @ Y_train

Since this ${\bf a}$ is based only on the training data, then it is only an estimate of what the relationship between ${\bf X}$ and ${\bf y}$ should be.  We'll see how accurate this estimated relationship is.

In [ ]:
Y_pred = np.matrix(X_train)*np.matrix(a)
test_predictions = np.matrix(X_test)*np.matrix(a)

In [ ]:
np.linalg.norm(Y_train - Y_pred)/np.linalg.norm(Y_train)

That's okay...

In [ ]:
np.linalg.norm(test_predictions - Y_test)/np.linalg.norm(Y_test)